In [13]:
import numpy as np

In [14]:
class ART_1():
    #初始化内星权向量矩阵B（C层向R层传入R的权值）和外星权向量矩阵T（R层反馈给C层）
    def set_B0_T0(self, n):
        b = 1 / (1 + n)
        B = []
        T = []
        for i in range(n):
            B.append(b)
            T.append(1)
        #numpy的mat函数将向量转化为矩阵
        B = np.mat(B)
        T = np.mat(T)
        #然后转置
        B = B.T
        T = T.T
        return B, T

    #竞争函数,选取R层竞争获胜的向量
    def Competitor(self, B, x):
        x = np.mat(x)#x(1,25)
        J = B.shape[1] #B(25,1)
        competition = []
        for j in range(J):
            compe = x * B[:, j]#x(1,25) B(25,1) compe(1,1)
            competition.append(compe[0, 0])
        competition = np.array(competition)
        j_order = np.argsort(-competition) #按大小排序,返回索引值
        j_order = list(j_order) #后面要用j_order循环所以又转回List类型
        return j_order

    
    #匹配，竞争成功的R层节点，通过外星权向量和x匹配，ruo0是阈值
    def Match(self, T, x, j, rou0):
        N0 = sum(x)
        x = np.mat(x)
        N = x * T[:, j]
        N = N[0, 0]
        rou = N/N0
        if rou >= rou0:
            matchBOOL = True
        else:
            matchBOOL = False
        return matchBOOL

    # 增加新节点
    def ADD_BT(self, B, T, x, n):
        b = x
        b = np.mat(b).T
        B = np.hstack((B,b))
        L=[]
        for i in range(len(x)):
            L.append(1)
        L = np.mat(L).T
        T = np.hstack((T, L))
        return B, T
    
    #分组
    def Group(self, B, T, x, j_order, Groups, rou0):
        success = 0
        n =len(x)
        jj = B.shape[1]
        for j in j_order:
            matchBooL = self.Match(T, x, j, rou0)#第一组True
            if matchBooL:
                B, T = self.AdjustBT(B, T, x, j)
                jj = j + 1
                if jj not in Groups:
                    Groups[jj] = [x]
                else:
                    Groups[jj].append(x)
                print(x, '分组成功，分在{}组'.format(jj))
                success = 1
                break
        if success == 0:
            B, T = self.ADD_BT(B, T, x, jj-1)
            jj = B.shape[1]
            B, T = self.AdjustBT(B, T, x, jj-1)
            Groups[jj] = [x]
            print(x, '分组成功，分在{}组'.format(jj))
        return B, T, Groups

    
    #匹配成功后，更新内外星向量权值
    def AdjustBT(self, B, T, x, j):
        m = len(x)
        x = np.mat(x)
        xigama_TX = x * T[:, j] #x(1,25) T(25,1) ,xigama(1,1)
        xigama_TX = xigama_TX[0, 0]
        new_b = 1 / (0.5 + xigama_TX)
        for i in range(m):
            if B[i, j].any() == 1:
                B[i, j] = new_b #更新内星权向量
        T[:, j] = np.multiply(T[:,j], x.T)#更新外星权向量
        return B, T

In [15]:
rou0 = 0.95
# X = [[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
#      [1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1],
#     [1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1],
#     [1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1]]

X=[
    [0,1,1,1,0,1,0,0,0,1,1,1,1,1,1,1,0,0,0,1,1,0,0,0,1],
    [1,1,1,1,0,1,0,0,0,1,1,1,1,1,0,1,0,0,0,1,1,1,1,1,0],
    [1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,1,1,1],
    [1,1,1,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,1,1,1,0,0],
    [1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,0,0,0,1,1,0,0,0,1],
    [0,0,1,0,0,0,1,0,1,0,1,1,1,1,1,1,0,0,0,1,1,0,0,0,1],
    [1,1,1,1,1,1,0,0,0,1,1,1,1,1,0,1,0,0,0,1,1,1,1,1,1],
    [1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1],
    [1,1,1,1,1,1,0,0,0,1,1,0,0,0,0,1,0,0,0,1,1,1,1,1,1],
    [0,1,1,1,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,1,1,1],
    [1,1,1,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,1,1,1,1,0],
    [1,1,1,1,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,1,1,1,1,0]
]

In [16]:
n = len(X) #输入四类模式
m = len(X[0]) #取模式的维数
art = ART_1()
#初始化分组
Groups = {}
#初始化B，T
B, T = art.set_B0_T0(m)

In [17]:
#对每一个x进行判断归类
for x in X:
    j_order = art.Competitor(B,x)
    B, T, Groups = art.Group(B, T, x, j_order, Groups, rou0)

[0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1] 分组成功，分在1组
[1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0] 分组成功，分在2组
[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1] 分组成功，分在3组
[1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0] 分组成功，分在4组
[1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1] 分组成功，分在5组
[0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1] 分组成功，分在6组
[1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1] 分组成功，分在7组
[1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1] 分组成功，分在8组
[1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1] 分组成功，分在7组
[0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1] 分组成功，分在3组
[1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0] 分组成功，分在9组
[1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,